In [1]:
#initializing image data generator with rescaling
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
#initializing image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [4]:
#pre-process all the train images
rain_generator = train_data_gen.flow_from_directory('../DataSet/images/train/',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 25311 images belonging to 7 classes.


In [5]:
#pre-process all the test images
ain_generator = validation_data_gen.flow_from_directory('../DataSet/images/train/',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 25311 images belonging to 7 classes.
